# 🎯 Le Modèle de Heston Complet

## Introduction

Nous voici au cœur du sujet ! Le modèle de Heston (1993) est l'un des modèles les plus utilisés en finance quantitative pour modéliser la **volatilité stochastique**.

### 🎯 Objectifs
1. Comprendre la formulation mathématique du modèle
2. Explorer le processus CIR pour la variance
3. Implémenter la simulation complète
4. Analyser l'impact des paramètres

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
%matplotlib inline

## 1. Formulation du Modèle de Heston

### 📐 Les Équations

Le modèle de Heston est défini par deux équations différentielles stochastiques (EDS) couplées :

$$\begin{cases}
dS_t = \mu S_t \, dt + \sqrt{v_t} \, S_t \, dW^S_t \\
dv_t = \kappa(\theta - v_t) \, dt + \sigma_v \sqrt{v_t} \, dW^v_t
\end{cases}$$

Avec : $\text{Corr}(dW^S_t, dW^v_t) = \rho \, dt$

### 📊 Les Paramètres

**Pour le prix $S_t$ :**
- $S_0$ : Prix initial de l'actif
- $\mu$ : Drift (rendement moyen espéré)
- $v_t$ : Variance instantanée (stochastique !)

**Pour la variance $v_t$ (processus CIR) :**
- $v_0$ : Variance initiale
- $\kappa$ : Vitesse de retour à la moyenne (mean reversion speed)
- $\theta$ : Variance de long terme (level)
- $\sigma_v$ : Volatilité de la volatilité (vol of vol)

**Corrélation :**
- $\rho$ : Corrélation entre les deux browniens (typiquement négatif : -0.8 à -0.5)

### 💡 Intuition de chaque paramètre

#### Le processus de variance (CIR)
$$dv_t = \underbrace{\kappa(\theta - v_t) dt}_{\text{Retour à la moyenne}} + \underbrace{\sigma_v \sqrt{v_t} dW^v_t}_{\text{Choc aléatoire}}$$

- Si $v_t < \theta$ → le drift est positif → $v_t$ tend à **augmenter**
- Si $v_t > \theta$ → le drift est négatif → $v_t$ tend à **diminuer**  
- $\kappa$ : à quelle vitesse revient-on vers $\theta$ ?
- $\sigma_v$ : amplitude des fluctuations de la variance

### 🔑 Condition de Feller

Pour garantir que $v_t > 0$ toujours, on impose :
$$2\kappa\theta > \sigma_v^2$$

Interprétation : La force de rappel vers $\theta$ doit être assez forte pour empêcher $v_t$ d'atteindre zéro.

## 2. Implémentation du Modèle de Heston

In [ ]:
class HestonModel:
    """
    Simulateur du modèle de Heston avec explication pédagogique
    """
    def __init__(self, S0, V0, mu, kappa, theta, sigma_v, rho):
        self.S0 = S0          # Prix initial
        self.V0 = V0          # Variance initiale  
        self.mu = mu          # Drift du prix
        self.kappa = kappa    # Vitesse de retour à la moyenne
        self.theta = theta    # Variance de long terme
        self.sigma_v = sigma_v  # Vol of vol
        self.rho = rho        # Corrélation
        
        # Vérifier condition de Feller
        feller_condition = 2 * kappa * theta
        if feller_condition <= sigma_v**2:
            print(f"⚠️  Condition de Feller violée: 2κθ={feller_condition:.4f} ≤ σ²={sigma_v**2:.4f}")
            print("   La variance peut atteindre zéro!")
        else:
            print(f"✅ Condition de Feller satisfaite: 2κθ={feller_condition:.4f} > σ²={sigma_v**2:.4f}")
    
    def simulate(self, T=1.0, N=1000, n_paths=1, seed=None):
        """
        Simule des trajectoires du modèle de Heston
        Méthode : Euler-Maruyama avec correction de variance
        """
        if seed is not None:
            np.random.seed(seed)
        
        dt = T / N
        sqrt_dt = np.sqrt(dt)
        t = np.linspace(0, T, N+1)
        
        # Initialisation
        S = np.zeros((N+1, n_paths))
        V = np.zeros((N+1, n_paths))
        S[0] = self.S0
        V[0] = self.V0
        
        # Précalcul pour corrélation
        rho_comp = np.sqrt(1 - self.rho**2)
        
        for i in range(N):
            # Générer browniens corrélés
            Z1 = np.random.standard_normal(n_paths)
            Z2 = np.random.standard_normal(n_paths)
            
            dW_S = sqrt_dt * Z1
            dW_v = sqrt_dt * (self.rho * Z1 + rho_comp * Z2)
            
            # Schéma de variance : méthode de la valeur absolue
            # (pour éviter variance négative)
            V_current = np.maximum(V[i], 0)  # Assurer V > 0
            sqrt_V = np.sqrt(V_current)
            
            # Mise à jour de la variance (CIR)
            V[i+1] = V[i] + self.kappa * (self.theta - V_current) * dt \
                     + self.sigma_v * sqrt_V * dW_v
            V[i+1] = np.maximum(V[i+1], 0)  # Correction : forcer V > 0
            
            # Mise à jour du prix
            S[i+1] = S[i] * np.exp((self.mu - 0.5*V_current) * dt 
                                    + sqrt_V * dW_S)
        
        return t, S, V
    
    def plot_paths(self, t, S, V, n_display=5):
        """Visualise les trajectoires"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))
        
        # Prix
        for i in range(min(n_display, S.shape[1])):
            ax1.plot(t, S[:, i], alpha=0.7, linewidth=2)
        ax1.axhline(y=self.S0, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Prix initial')
        ax1.set_title('Trajectoires du Prix S(t)', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Temps')
        ax1.set_ylabel('Prix')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Variance
        for i in range(min(n_display, V.shape[1])):
            ax2.plot(t, V[:, i], alpha=0.7, linewidth=2)
        ax2.axhline(y=self.theta, color='green', linestyle='--', linewidth=2, 
                   label=f'Variance long terme θ={self.theta:.4f}')
        ax2.axhline(y=self.V0, color='red', linestyle='--', linewidth=2, alpha=0.5,
                   label=f'Variance initiale V₀={self.V0:.4f}')
        ax2.set_title('Trajectoires de la Variance v(t)', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Temps')
        ax2.set_ylabel('Variance')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Test avec paramètres réalistes (Bitcoin)
print("=" * 80)
print("SIMULATION DU MODÈLE DE HESTON - Exemple Bitcoin")
print("=" * 80)

# Paramètres calibrés type crypto
model = HestonModel(
    S0=50000,        # Prix Bitcoin actuel
    V0=0.16,         # Variance initiale (40% volatilité annuelle)
    mu=0.15,         # 15% drift annuel
    kappa=2.0,       # Retour rapide à la moyenne
    theta=0.16,      # Variance long terme = initiale
    sigma_v=0.5,     # Vol of vol modérée
    rho=-0.7         # Corrélation négative (effet de levier)
)

t, S, V = model.simulate(T=1.0, N=252, n_paths=5, seed=42)
model.plot_paths(t, S, V)

## 3. Impact des paramètres - Expériences

In [ ]:
# Étude de l'impact de la corrélation ρ
rhos = [-0.9, -0.5, 0, 0.5, 0.9]
n_sims = 1000
T = 1.0
N = 252

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

base_params = {
    'S0': 100, 'V0': 0.04, 'mu': 0.05,
    'kappa': 2.0, 'theta': 0.04, 'sigma_v': 0.3
}

for idx, rho in enumerate(rhos):
    model = HestonModel(**base_params, rho=rho)
    t, S, V = model.simulate(T=T, N=N, n_paths=n_sims, seed=42)
    
    # Calculer log-rendements
    log_returns = np.log(S[-1, :] / base_params['S0'])
    
    # Histogramme
    axes[idx].hist(log_returns, bins=50, alpha=0.7, edgecolor='black', density=True)
    axes[idx].axvline(np.mean(log_returns), color='red', linestyle='--', linewidth=2,
                     label=f'Moyenne: {np.mean(log_returns):.3f}')
    axes[idx].set_title(f'ρ = {rho} | Skewness = {stats.skew(log_returns):.3f}',
                       fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Log-rendement')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

fig.delaxes(axes[5])
plt.suptitle('Impact de la corrélation ρ sur la distribution des rendements',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("📊 Observations :")
print("  • ρ < 0 : Distribution asymétrique NÉGATIVE (queue gauche)")
print("  • ρ = 0 : Distribution presque symétrique")
print("  • ρ > 0 : Distribution asymétrique POSITIVE (queue droite)")
print("\n✅ En pratique, ρ < 0 pour les actifs risqués (effet de levier)")

In [ ]:
# Impact du vol of vol (sigma_v)
sigma_vs = [0.1, 0.3, 0.5, 0.7]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, sig_v in enumerate(sigma_vs):
    model = HestonModel(**base_params, rho=-0.7, sigma_v=sig_v)
    t, S, V = model.simulate(T=1.0, N=252, n_paths=10, seed=idx)
    
    # Tracer variance uniquement
    for i in range(10):
        axes[idx].plot(t, V[:, i], alpha=0.6)
    
    axes[idx].axhline(y=base_params['theta'], color='red', linestyle='--',
                     linewidth=2, label='θ (long terme)')
    axes[idx].set_title(f'σᵥ = {sig_v} | Volatilité de la variance',
                       fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Temps')
    axes[idx].set_ylabel('Variance v(t)')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Impact de σᵥ (vol of vol) sur la dynamique de variance',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("📊 Observations :")
print("  • σᵥ faible (0.1) : Variance reste proche de θ (peu de fluctuations)")
print("  • σᵥ élevé (0.7) : Variance très volatile (clusters de volatilité)")
print("\n✅ σᵥ élevé → captures mieux les crises (pics de volatilité)")

## 4. Comparaison avec Black-Scholes

### 💡 Black-Scholes = Heston avec volatilité constante

Si on fixe $\sigma_v = 0$ (pas de vol of vol), alors $v_t = \theta$ constant → on retrouve Black-Scholes !

In [ ]:
# Comparaison directe
np.random.seed(42)

# Black-Scholes (volatilité constante)
vol_BS = np.sqrt(0.04)  # 20% annuelle
S_BS = np.zeros((253, 1000))
S_BS[0] = 100
dt = 1/252
for i in range(252):
    Z = np.random.standard_normal(1000)
    S_BS[i+1] = S_BS[i] * np.exp((0.05 - 0.5*vol_BS**2)*dt + vol_BS*np.sqrt(dt)*Z)

# Heston
model_heston = HestonModel(S0=100, V0=0.04, mu=0.05, kappa=2.0, 
                           theta=0.04, sigma_v=0.4, rho=-0.7)
t, S_Heston, V_Heston = model_heston.simulate(T=1.0, N=252, n_paths=1000, seed=42)

# Distributions finales
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histogrammes
ax1.hist(S_BS[-1], bins=50, alpha=0.6, label='Black-Scholes', edgecolor='black', density=True)
ax1.hist(S_Heston[-1], bins=50, alpha=0.6, label='Heston', edgecolor='black', density=True)
ax1.set_title('Distribution des prix finaux S(T)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Prix final')
ax1.set_ylabel('Densité')
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# Q-Q plot
stats.probplot(np.log(S_BS[-1]/100), dist="norm", plot=ax2)
ax2.get_lines()[0].set_marker('o')
ax2.get_lines()[0].set_markersize(3)
ax2.get_lines()[0].set_label('Black-Scholes')

stats.probplot(np.log(S_Heston[-1]/100), dist="norm", plot=ax2)
ax2.get_lines()[2].set_marker('s')
ax2.get_lines()[2].set_markersize(3)
ax2.get_lines()[2].set_color('orange')
ax2.get_lines()[2].set_label('Heston')

ax2.set_title('Q-Q Plot (test de normalité)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Résultats :")
print(f"  Black-Scholes - Skewness: {stats.skew(np.log(S_BS[-1]/100)):.3f}")
print(f"  Heston        - Skewness: {stats.skew(np.log(S_Heston[-1]/100)):.3f}")
print(f"  Black-Scholes - Kurtosis: {stats.kurtosis(np.log(S_BS[-1]/100)):.3f}")
print(f"  Heston        - Kurtosis: {stats.kurtosis(np.log(S_Heston[-1]/100)):.3f}")
print("\n✅ Heston capture mieux les queues épaisses (fat tails) des distributions réelles")

## 🎯 Résumé

### Ce que nous avons appris :

1. **Modèle de Heston** : 2 EDS couplées pour prix + variance stochastique
2. **Processus CIR** : retour à la moyenne pour la variance
3. **Paramètres clés** :
   - $\rho$ : asymétrie de la distribution (effet de levier)
   - $\sigma_v$ : clusters de volatilité
   - $\kappa$ : vitesse d'ajustement
4. **Avantages vs Black-Scholes** : queues épaisses, smile de volatilité

### 📖 Prochain notebook : Simulations Monte Carlo

Nous allons :
- Générer des milliers de scénarios
- Calculer des statistiques et probabilités
- Estimer des percentiles
- Préparer pour le rapport final !

---

**Continuez vers 04_Simulations_Monte_Carlo.ipynb** 🚀